In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("assignment4.ipynb")

# Final Project <a class='tocSkip'>
    
## PSTAT 134/234 (Winter 2023) <a class='tocSkip'>
    
## Collaboration Policy <a class='tocSkip'>

Data science is a collaborative activity. While you may talk with others about
the homework, we ask that you **write your solutions individually**. If you do
discuss the assignments with others please **include their names** at the top
of your notebook.

**Collaborators**: *list collaborators here*

In [2]:
%xmode Verbose

Exception reporting mode: Verbose


<!-- BEGIN QUESTION -->

## Question 1: Using Linear Algebra for Optimization
In recommender system module, low-rank matrix factorization was used to execute latent factor modeling of movie ratings data.

Specifically, we calculated matrices $U$ and $V$ to solve the following optimization problem (if all ratings were given):
$$
\begin{aligned}
\min_{U,V} f(U,V) &= \min_{U,V} \|R - V U^T\|_F^2
=\min_{U,V} \left\{ \sum_{m=1}^M\sum_{i=1}^I I_{mi}(r_{mi} - v_m u_i^T)^2 \right\},
\end{aligned}
$$
where
$$
\begin{aligned}
I_{mi} = \begin{cases}
1 \text{, if $r_{mi}$ is observed}\\
0 \text{, if $r_{mi}$ is missing.}\\
\end{cases}
\end{aligned}
$$

The best $U$ and $V$ were calculated iteratively by improving on current estimates:
$$
\begin{aligned}
u_i^{\text{new}} &= u_i + 2\alpha(r_{mi} -  v_m u_i^T)\cdot v_m\\
v_m^{\text{new}} &= v_m + 2\alpha(r_{mi} -  v_m u_i^T)\cdot u_i,
\end{aligned}
$$
where $\alpha$ is the step-size that is to be chosen by the user. (We won't discuss the role in this class, but treat it as an arbitrary, but given, parameter) 

We can make calculating the updates more efficient by calculating them with matrix operations. For example, instead of calculating each deviation $\gamma_{mi} = r_{mi} - v_m u_i^T$ separately for all $m=1,2,\dots,M$ and $i=1,2,\dots,I$, matrix $\Gamma$ of all deviations can be computed together using matrix operation _(verify for yourself)_:
$$\Gamma = R - VU^T$$

Similarly, updating $U$ and $V$ can be combined into matrix calculations which makes the optimization procedure more efficient.

First, note that updates for $u_i$, $i=1,2,\dots,I$ can be rewritten as
$$
\begin{aligned}
u_1^{\text{new}} &= u_1 + 2\alpha\gamma_{m1}\cdot v_m\\
u_2^{\text{new}} &= u_2 + 2\alpha\gamma_{m2}\cdot v_m\\
\vdots\quad &\qquad\qquad\vdots\\
u_I^{\text{new}} &= u_I + 2\alpha\gamma_{mI}\cdot v_m.
\end{aligned}
$$
Stacking all $I$ equations into a matrix form, 
$$
\begin{aligned}
U^{\text{new}} &= U + 2\alpha\Gamma_{m-}^T v_m,
\end{aligned}
$$
where $\Gamma_{m-}$ is the $m$-th row of $\Gamma$ (use the notation $\Gamma_{-i}$ for the $i$-th column). When evaluating $U^{\text{new}}$, the latest updated values of $U$, $V$, and $\Gamma$ are used.

Note that there are $M$ such update equations (one for each $m=1,2,\dots,M$) that can also be combined into one matrix update equation involving matrices $U$, $V$, $\Gamma$ and scalars. As stated earlier, since $\alpha$ is assumed to be an arbitrary step-size parameter, we can replace $\alpha/M$ with $\alpha$.

### Question 1a: Using Linear Algebra for Optimization

Complete the following update equations:
$$
\begin{aligned}
U^{\text{new}} &= U + 2\alpha[\text{some function of }\Gamma][\text{some function of }V]\\
V^{\text{new}} &= V + 2\alpha[\text{some function of }\Gamma][\text{some function of }U]
\end{aligned}
$$

<!--
BEGIN QUESTION
name: q1a
manual: true
points: 4
-->

$$
\begin{aligned}
U^{\text{new}} &= U + 2\alpha\Gamma^T V\\
V^{\text{new}} &= V + 2\alpha\Gamma U
\end{aligned}
$$

<!-- END QUESTION -->

### Question 1b: Implementing Updates

In this problem, you will implement the updates calculated in the previous problem. Define the following three functions:

* `update_G(R, U, V)`: computes deviation $R-VU^T$
* `update_U(G, U, V, alpha=0.01)`: calculates update $U^{\text{new}}$
* `update_V(G, U, V, alpha=0.01)`: calculates update $V^{\text{new}}$

Each function should only be one line of matrix operations. Three functions is to be applied sequentially, using the most up-to-date estimates of $G$, $U$, and $V$.

Since some elements of `R` are `np.nan` for any missing ratings, `update_U` and `update_V` functions need to be adjusted by using `numpy.nan_to_num` function where appropriate. The function `numpy.nan_to_num` will let you replace `NaN` to some number, so that missing ratings do not interfere with updates.

<!--
BEGIN QUESTION
name: q1b
manual: false
points: 4
-->

In [3]:
import numpy as np
import pandas as pd

def update_G(R_, U_, V_):
    index_nan_R = np.transpose(np.where(np.isnan(R_)))
    G_ = np.nan_to_num(R_) - np.dot(V_, np.transpose(U_))
    for i in index_nan_R:
        G_[tuple(i)] = np.nan
    return G_

def update_U(G_, U_, V_, alpha=0.01):
    U_new = U_ + (2 * alpha * np.dot(np.nan_to_num(np.transpose(G_)), np.nan_to_num(V_))) 
    return U_new

def update_V(G_, U_, V_, alpha=0.01):
    V_new = V_ + (2 * alpha * np.dot(np.nan_to_num(G_), np.nan_to_num(U_)))
    return V_new

# small test to help debug (keep intact)
np.random.seed(1)

M_ = 5
I_ = 3
K_ = 2

R_ = np.random.rand(M_, I_).round(1)
R_[0, 0] = R_[3, 2] = np.nan
U_ = np.random.rand(I_, K_).round(1)
V_ = np.random.rand(M_, K_).round(1)
G_ = update_G(R_, U_, V_)

In [4]:
grader.check("q1b")

q1b passed!

### Question 1c: Construct Optimization Algorithm

Combine the above functions to implement the optimization algorithm to iteratively compute $U$ and $V$.

But, first, here are functions that will calculate RMSE and quantify the maximum update (in absolute value) made by `update_U` and `update_V` after they are called.

In [5]:
def rmse(X):
    """
    Computes root-mean-square-error, ignoring nan values
    """
    return np.sqrt(np.nanmean(X**2))

def max_update(X, Y, relative=True):
    """
    Compute elementwise maximum update
    
    parameters:
    - X, Y: numpy arrays or vectors
    - relative: [True] compute relative magnitudes
    
    returns
    - maximum difference between X and Y (relative to Y) 
    
    """
    if relative:
        updates = np.nan_to_num((X - Y)/Y)
    else:
        updates = np.nan_to_num(X - Y)
            
    return np.linalg.norm(updates.ravel(), np.inf)

A template for the optimization algorithm is given below. Fill-in the missing portions to complete the algorithm.

<!--
BEGIN QUESTION
name: q1c1
manual: false
points: 4
-->

In [6]:
def compute_UV(Rdf, K=5, alpha=0.01, max_iteration=5000, diff_thr=1e-3):

    R = Rdf.values
    Rone = pd.DataFrame().reindex_like(Rdf).replace(np.nan, 1) # keep data frame metadata

    M, I = R.shape            # number of movies and users
    U = np.random.rand(I, K)  # initialize with random numbers
    V = np.random.rand(M, K)  # initialize with random numbers
    G = update_G(R, U, V)     # calculate residual

    track_rmse = []
    track_update = []
    for i in range(0, max_iteration): 
        
        Unew = update_U(G, U, V, alpha=alpha)
        Gnew = update_G(R, Unew, V)

        Vnew = update_V(Gnew, Unew, V, alpha=alpha)
        Gnew = update_G(R, Unew, Vnew)

        track_rmse += [{
            'iteration':i, 
            'rmse': rmse(Gnew),
            'max residual change': max_update(Gnew, G, relative=False)
        }]
        track_update += [{
            'iteration':i, 
            'max update':max(max_update(Unew, U), max_update(Vnew, V))
        }]

        U = Unew
        V = Vnew
        G = Gnew
        
        if track_update[-1]['max update'] < diff_thr:
            break
        
    track_rmse = pd.DataFrame(track_rmse)
    track_update = pd.DataFrame(track_update)
    
    kindex = pd.Index(range(0, K), name='k')
    U = pd.DataFrame(U, index=Rdf.columns, columns=kindex)
    V = pd.DataFrame(V, index=Rdf.index, columns=kindex)
    
    return {
        'U':U, 'V':V,
        'rmse': track_rmse,
        'update': track_update
    }
 
Rsmall = pd.read_pickle('data/ratings_stacked_small.pkl').unstack()

np.random.seed(134) # set seed for tests
output1 = compute_UV(Rsmall, K=10, alpha=0.001)

In [7]:
grader.check("q1c1")

q1c1 passed!

Running the function on a different sized problem to check if `compute_UV` adapts to changing problem sizes.
There is nothing new to do here

<!--
BEGIN QUESTION
name: q1c2
manual: false
points: 4
-->

In [8]:
# These tests should pass if `compute_UV` works properly
np.random.seed(134) # set seed for tests
output2 = compute_UV(Rsmall.iloc[:7, :5], K=8)

In [9]:
grader.check("q1c2")

q1c2 passed!

### Question 1d: Interpret Diagnostic Plots

Following figures tell us if the optimization algorithm is working properly.

In [10]:
import altair as alt
logscale = alt.Scale(type='log', base=10)
fig_rmse = \
    alt.Chart(output1['rmse'])\
    .mark_line()\
    .encode(
        x='iteration:Q', 
        y=alt.Y('rmse:Q', scale=logscale)
    )
fig_max_residual_change = \
    alt.Chart(output1['rmse'])\
    .mark_line()\
    .encode(
        x='iteration:Q', 
        y=alt.Y('max residual change:Q', scale=logscale)
    )
fig_updates = \
    alt.Chart(output1['update'])\
    .mark_line()\
    .encode(
        x='iteration:Q', 
        y=alt.Y('max update:Q', scale=logscale)
    )
alt.vconcat(
    fig_rmse | fig_max_residual_change,
    fig_updates 
)

alt.VConcatChart(...)

<!-- BEGIN QUESTION -->

By referring back to the function used to calculate the quantities in each figure, describe what each figure is showing and interpret the behavior of the optimization algorithm.

<!--
BEGIN QUESTION
name: q1d
manual: true
points: 4
-->

As we can see above, all the plots essentially give us some insight on how well our optimization algorithm is doing. The **RMSE Plot** as we clearly see has a very strong decreasing trend as iteration increases. Between 0 and 500 iterations we see a sharpe decline in RMSE that drops from $\approx$ 1.75 to $\approx$ .175. This indicates that with more iterations of our algorithm, we can drastically reduce error within just 500 iterations, and even moreso within the first 5000 as well. The **Max Residual Change Plot** shows a decreasing trend as well that further decreases as iteration increases. This essentially, means that as the number of iterations increase, there will be a smaller range of differences between residuals. This means, that our algorithm is predicting values much closer to the observed values as we increase the number of iterations. The **Max Update Plot** displays how our U and V matricies change in value anywhere with each update. From the looks of our plot, it seems we have a range of change in the updated value that extends from $\approx$ 0.001 to $\approx$ 100. As we will also notice, is that over time, although there are occasional large spikes in the change of values after updating, we still see an overall decreasing trend that drops from $\approx$ 1 to $\approx$ 0.001.

<!-- END QUESTION -->



### Question 1e: Analyze Large Dataset

Following code will analyze a larger dataset:

In [11]:
# run on larger dataset: ratings for 100 movies 
Rbig = pd.read_pickle('data/ratings_stacked.pkl').unstack().iloc[:100]

np.random.seed(14) # set seed for tests
output4 = compute_UV(Rbig, K=5, alpha=0.001, max_iteration=500)

Rhatbig = output4['V']@output4['U'].T

In [12]:
fit_vs_obs = pd.concat([
    Rhatbig.rename(columns={'rating':'fit'}),
    Rbig.rename(columns={'rating':'observed'}),
], axis=1).stack().dropna().reset_index()[['fit','observed']]

fit_vs_obs = fit_vs_obs.iloc[np.random.choice(len(fit_vs_obs), 5000)]

alt.Chart(fit_vs_obs).transform_density(
    density='fit',
    bandwidth=0.01,
    groupby=['observed'],
    extent= [0, 6]
).mark_bar().encode(
    alt.X('value:Q'),
    alt.Y('density:Q'),
    alt.Row('observed:N')
).properties(width=800, height=50)

alt.Chart(...)

<!-- BEGIN QUESTION -->

Consider the above plot. By reading the code, comment on what the plot is illustrating. What happens when you add `counts=True` to `transform_density`? What can you conclude?

<!--
BEGIN QUESTION
name: q1e
manual: true
points: 4
-->

The plot above is computing a one-dimensional kernel density estimation, illustrating the density (y-axis) of the predicted rating values (x-axis) using a histogram plot. In this plot, we have propability estimates at each value of the x-axis, which again represent the rating value predicted by our algorithm.

Next, as we can see below, when adding the argument `counts=True`, our density axis is rescaled from the previous y-axis range of $[0, 1]$ to $[0, 2000]$. When setting `counts=True` we are no longer computing probability estimates of each predicted value rating, but rather we are now computing a smoothed count plot that displays the amount of total number of instances that each value of the x-axis was predicted. This is why the scale of our plots have changed, because we had $2,000$ iterations.

In [13]:
alt.Chart(fit_vs_obs).transform_density(
    density='fit',
    bandwidth=0.01,
    groupby=['observed'],
    extent= [0, 6],
    counts=True
).mark_bar().encode(
    alt.X('value:Q'),
    alt.Y('density:Q'),
    alt.Row('observed:N')
).properties(width=800, height=50)

alt.Chart(...)

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

### Question 1f: Make Recommendation

What movies would you recommend to `user id` 601? Do you see any similarities to movies the user rated high?

<!--
BEGIN QUESTION
name: q1f
manual: true
points: 4
-->

As we can see from the output below, after computing our `Rhatbig` matrix, I would most likely recommend the top 5 movies outputs to User 601.

In [14]:
Rhatbig.iloc[:, 600].sort_values(ascending=False)

movie id  movie title                            
13        Mighty Aphrodite (1995)                    4.000717
47        Ed Wood (1994)                             3.910924
100       Fargo (1996)                               3.851010
48        Hoop Dreams (1994)                         3.844658
19        Antonia's Line (1995)                      3.792223
                                                       ...   
94        Home Alone (1990)                          0.854282
62        Stargate (1994)                            0.334891
34        Doom Generation, The (1995)                0.118788
78        Free Willy (1993)                          0.114008
35        Free Willy 2: The Adventure Home (1995)   -0.039283
Name: (rating, 601), Length: 100, dtype: float64

Also as we see below is the observed values of the movies that User 601 has been seen to rate high. By looking at these movies I would say the similarities I see between the top 5 highest rated movies is that movies rated between $3.0$ and $4.0$ are much more "adult" movies whereas movies rate a $5.0$ are more "childish" or fantasy films.

In [15]:
Rbig.iloc[:, 600].sort_values(ascending=False)

movie id  movie title                           
50        Star Wars (1977)                          5.0
91        Nightmare Before Christmas, The (1993)    5.0
64        Shawshank Redemption, The (1994)          4.0
87        Searching for Bobby Fischer (1993)        4.0
65        What's Eating Gilbert Grape (1993)        4.0
                                                   ... 
92        True Romance (1993)                       NaN
93        Welcome to the Dollhouse (1995)           NaN
94        Home Alone (1990)                         NaN
95        Aladdin (1992)                            NaN
97        Dances with Wolves (1990)                 NaN
Name: (rating, 601), Length: 100, dtype: float64

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

## Question 2: Improving the Model

### Question 2a: Logistic function 

Note the reconstructed ratings can be smaller than 1 and greater than 5. To confine ratings to between the allowed range, we can use the logistic function. Logistic function is defined as 
$$ h(x) = \frac{1}{1+e^{-x}}. $$
It is straightforward to show the derivative is 
$$ h'(x) = \frac{e^{-x}}{(1+e^{-x})^2} = h(x)(1-h(x)). $$
Therefore, we can rescale the ratings from $r_{mi}\in [1, 5]$ to $r_{mi}\in [0, 1]$. Then, we can find the best $U$ and $V$ to optimize the following:
$$ \min_{U,V} \| R - h(VU^T) \|_F^2 = \sum_{m,i} I_{mi}(r_{mi} - h(v_m u_i^T))^2, $$
where function $h$ is applied elementwise and 
$$
\begin{aligned}
I_{mi} = \begin{cases}
1 \text{, if $r_{mi}$ is observed}\\
0 \text{, if $r_{mi}$ is missing.}\\
\end{cases}
\end{aligned}
$$

Derive new update expressions for the new objective function.
<!--
BEGIN QUESTION
name: q2a
manual: true
points: 4
-->

$$
\begin{aligned}
U^{\text{new}} &= U + 2\alpha \times h'(VU^T) \Gamma_{m-}^T V\\
V^{\text{new}} &= V + 2\alpha \times h'(VU^T) \Gamma_{-i} U
\end{aligned}
$$

<!-- END QUESTION -->

### Quesiton 2b: Implementation

Implement the update functions in functions below.

<!--
BEGIN QUESTION
name: q2b1
manual: false
points: 4
-->

In [16]:
def logistic(x):
    """
    Evaluates logistic function
    
    """
    return 1/(1+np.exp(-x))

def update_logistic_G(R_, U_, V_):
    
    return R_ - logistic(V_@U_.T)

def update_logistic_U(G_, U_, V_, alpha=0.01):
    
    logisticVUT = logistic(V_@U_.T)              # estimated ratings
    grad = -2 * np.nan_to_num((G_*(1-logisticVUT)*logisticVUT).T) # gradient direction
    return U_ - (grad@V_*alpha)                     # gradient descent update from U_

def update_logistic_V(G_, U_, V_, alpha=0.01):
    
    logisticVUT = logistic(V_ @ U_.T)              # estimated ratings
    grad = -2 * np.nan_to_num(G_*(1-logisticVUT)*logisticVUT) # gradient direction
    return V_ - (grad@U_*alpha)                     # gradient descent update from V_

# small test to help debug (keep intact)
np.random.seed(1)

M_ = 5
I_ = 3
K_ = 2

R_ = np.random.rand(M_, I_).round(1)
R_[0, 0] = R_[3, 2] = np.nan
U_ = np.random.rand(I_, K_).round(1)
V_ = np.random.rand(M_, K_).round(1)
G_ = update_G(R_, U_, V_)

In [17]:
grader.check("q2b1")

q2b1 passed!

Now create a function `compute_logistic_UV` below:

<!--
BEGIN QUESTION
name: q2b2
manual: false
points: 4
-->

In [18]:
def compute_logistic_UV(Rdf, K=5, alpha=0.01, max_iteration=5000, diff_thr=1e-3):

    R = Rdf.values
    R = (R.copy()-1)/4         # map ratings to between 0 and 1
    Rone = pd.DataFrame().reindex_like(Rdf).replace(np.nan, 1) # keep data frame metadata

    M, I = R.shape                 # number of movies and users
    U = np.random.rand(I, K)-0.5   # initialize with random numbers
    V = np.random.rand(M, K)-0.5   # initialize with random numbers
    G = update_G(R, U, V)          # calculate residual

    track_rmse = []
    track_update = []
    for i in range(0, max_iteration): 
        
        Unew = update_logistic_U(G, U, V, alpha=alpha)
        Gnew = update_logistic_G(R, Unew, V)

        Vnew = update_logistic_V(Gnew, Unew, V, alpha=alpha)
        Gnew = update_logistic_G(R, Unew, Vnew)

        track_rmse += [{
            'iteration':i, 
            'rmse': rmse(Gnew),
            'max residual change': max_update(Gnew, G, relative=False)
        }]
        track_update += [{
            'iteration':i, 
            'max update':max(max_update(Unew, U), max_update(Vnew, V))
        }]

        U = Unew
        V = Vnew
        G = Gnew
        
        if track_update[-1]['max update'] < diff_thr:
            break
        
    track_rmse = pd.DataFrame(track_rmse)
    track_update = pd.DataFrame(track_update)
    
    kindex = pd.Index(range(0, K), name='k')
    U = pd.DataFrame(U, index=Rdf.columns, columns=kindex)
    V = pd.DataFrame(V, index=Rdf.index, columns=kindex)
    
    return {
        'U':U, 'V':V,
        'rmse': track_rmse,
        'update': track_update
    }

def logistic_rating(U_, V_):
    """
    converts the rating back to 1 to 5 rating
    """
    return( 4*logistic(V_@U_.T) + 1 )
    
np.random.seed(134) # set seed for tests
output3 = compute_logistic_UV(Rsmall, K=10, alpha=0.05)

In [19]:
grader.check("q2b2")

q2b2 passed!

### Question 2c: Analyze a Large Dataset

Following code will analyze a larger dataset:

In [20]:
# run on larger dataset: ratings for 100 movies 
Rbig = pd.read_pickle('data/ratings_stacked.pkl').unstack().iloc[:100]

np.random.seed(14) # set seed for tests
output4 = compute_logistic_UV(Rbig, K=5, alpha=0.05, max_iteration=500)

Rhatbig = logistic_rating(output4['U'], output4['V'])

In [21]:
Rhatbig.min()

        user id
rating  1          1.390898
        2          1.006746
        3          1.235549
        4          1.084232
        5          1.693782
                     ...   
        939        1.017401
        940        1.727798
        941        1.590295
        942        1.032400
        943        1.816917
Length: 943, dtype: float64

In [22]:
fit_vs_obs_2 = pd.concat([
    Rhatbig.rename(columns={'rating':'fit'}),
    Rbig.rename(columns={'rating':'observed'}),
], axis=1).stack().dropna().reset_index()[['fit','observed']]

fit_vs_obs_2 = fit_vs_obs_2.iloc[np.random.choice(len(fit_vs_obs_2), 5000)]

alt.Chart(fit_vs_obs_2).transform_density(
    density='fit',
    bandwidth=0.01,
    groupby=['observed'],
    extent= [0, 6]
).mark_bar().encode(
    alt.X('value:Q'),
    alt.Y('density:Q'),
    alt.Row('observed:N')
).properties(width=800, height=50)

alt.Chart(...)

<!-- BEGIN QUESTION -->

Consider the above plot. By reading the code, comment on what the plot is illustrating. How does this plot look different than part 1.e?

<!--
BEGIN QUESTION
name: q2c
manual: true
points: 4
-->

The first thing that is noticeable in this new plot, is the fact that our plot has a clear range of predicted values at the upper bound which seems to have a max value of $5$. By looking at the code, the reason why we have this change in visualization is due to the use the `logistic_rating()` function from part 2b which converts our rating matrix back to a $1.0$ to $5.0$ scale. We see this change most in the bottom most plot, which is not only very clearly left-skewed, but has again the sharp and appartent cut off at a value of $5.0$. This logistic function from question 2 that was created, not only led the histogram to change in the range of x values (value), but also change in overall shape since densities had to be re-computed when constricted between $1.0$ and $5.0$.

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

## (PSTAT 234) Question 3: Segmentation in latent factor space

Now that we have user matrix $U$ and movie matrix $V$, suppose we want to use the newly learned representation for an advertising campaign. 

Suppose you are leading the planning of an online advertising campaign and you have a fixed budget. With the budget, you can create 5 variations of an ad, and you want to create the variations based on a representative movie each group likes.

The advertisements will entice the viewer to sign up for a mailing list by offering a free poster. The goal of the advertising campaign is two fold: 

1. Get potential customers to sign up using their email address by offering a free poster among the 5 "representative" movies
2. Learn their user segment placement preference to use for the starting point for movie recommendations once they sign up

In order to achieve this goal, we want to 
1. Produce clusterings of users
2. Balance performance metric of clustering results and practical considerations to choose one of the clustering results.

We will tackle this step by step.

### Question 3a: Concatenate matrix factors and cluster

Entries in either matrix factors are just points in $k$-dimensional latent variable space. We will use both $U$ and $V$ for segmentation by combining them into one large clustering problem.

Once clusters are identified, you will qualitatively inspect the users and movies in the cluster and decide on a "representative" movie from each cluster.

Consider concatenating $U$ and $V$ into one large matrix. Since these matrices have arbitrary scaling, it would be a good idea to standardize the columns before concatenating them. Standardize $U$ and $V$ separately, then concatenate with numpy's `concatenate` method. Call this concatenated matrix, `UVstd`.

Apply hierarchical and K-means clustering methods on `UVstd`.  For each clustering method, identify 5 clusters. Compare the clustering results by applying three different [cluster validation metrics](https://scikit-learn.org/stable/modules/clustering.html#clustering-performance-evaluation) to evaluate the clustering performance. 

Which cluster performance metrics can you use? Do we have true labels? Does one performance metric seem to clearly be better than another? Why would you choose one metric over another? What interpretation, if any, does each metric have in the context of our problem? Explain.

<!--
BEGIN QUESTION
name: q3a
manual: true
points: 20
-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

### Question 3b: Making decisions


To make actionable decisions, there are practical considerations to take into account.

1. How will you choose a "representative" movie from each cluster?
2. How many of each poster do you estimate you will need? Assume the ad campaign will serve 10 million users and 0.01% people will respond. What other assumption do you need to make?
3. Which clustering method will you use as the final method?


<!--
BEGIN QUESTION
name: q3b
manual: true
points: 20
-->


_Type your answer here, replacing this text._

<!-- END QUESTION -->



_Cell Intentionally Blank_

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [25]:
grader.check_all()

q1b passed!

q1c1 passed!

q1c2 passed!

q2b1 passed!

q2b2 passed!

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [24]:
# Save your notebook first, then run this cell to export your submission.
grader.export()